# Les films Disney+
Dans ce projet consiste à scraper les données du site **allociné**

Pour la preparation de la cérémonie de recompense, On veut faire des projection de films disney+ au cinema pour les enfants afin de choisir le meilleur film en fonction de la **Date, le critique des spectateur, et l'age autorisé**. Sur le site, les résultats de recherche des films s’affichent sur environ 41 pages. Chaque page regroupe 12 annonces. Voici les adresses des trois premières pages :
- https://www.allocine.fr/disney/film/tous/?page=1
- https://www.allocine.fr/disney/film/tous/?page=2
- https://www.allocine.fr/disney/film/tous/?page=3

On a remarqué qu’elles sont quasiment identiques. Afin d’obtenir une liste des 41pages web à consulter, il suffit juste de modifier le dernier chiffre indiquant le numéro de la page. On peut le faire avec la fonction suivante :

#### Analyser le code html de la page web en utilisant les bibliothèques

La première étape est d’importer les bibliothèques qu'on utilisera pour scraper une page.
- le module **get** de la bibliothèque **requests** nous permettra à chaque page
- le module **BeautifulSoup** de **bs4** nous aidera à analyser et gérer le html.

La prochaine bibliothèque que nous importerons est urllib qui permet d’établir une connexion avec la page web. Aussi, les résultats seront inscrits dans un CSV, ce qui fait que nous avons aussi besoin d’importer la bibliothèque csv. Comme alternative, la bibliothèque json pourrait être utilisée à la place.

In [34]:
import requests
from requests import get
from bs4 import BeautifulSoup
import csv

In [44]:
url='https://www.allocine.fr/disney/film/tous/?page=1'

In [56]:
reponse=get(url)
parser=BeautifulSoup(reponse.text,'html.parser')
films=parser.find_all('li',class_='mdl')
print(type(films))
print("le nombre de films sur la première page est=",len(films))


<class 'bs4.element.ResultSet'>
le nombre de films sur la première page est= 12


## Scraper les informations du premier film sur Disney+
- Le nom du film
- La date de sortie
- L'age requis pour regarder le film
- La critique des spectateurs 

In [115]:
first_movie=films[0]
first_movie

<li class="mdl">
<div class="card entity-card entity-card-list cf">
<figure class="thumbnail">
<span class="ACrL3ZACrpZGVvL3BsYXllcl9nZW5fY21lZGlhPTE5NTkwNTI4JmNmaWxtPTI1MTM1NC5odG1s thumbnail-container thumbnail-link" title="Bande-annonce Soul">
<img alt="Bande-annonce Soul" class="thumbnail-img" height="420" src="https://fr.web.img4.acsta.net/c_310_420/pictures/20/10/15/11/09/5088324.jpg" width="310"/>
<span class="ico-play ico-play-yellow">
<span class="ico-play-inner"></span>
<i class="ico-play-arrow">
<i class="arrow"></i>
</i>
</span>
</span>
</figure>
<div class="meta">
<h2 class="meta-title">
<a class="meta-title-link" href="/film/fichefilm_gen_cfilm=251354.html">Soul</a>
</h2>
<div class="meta-body">
<div class="meta-body-item meta-body-info">
<span class="date">25 décembre 2020</span>
<span class="spacer">/</span>
                    
                                                            
                                            1h 40min
                             

In [154]:
titre=first_movie.a
titre.text

'Soul'

In [62]:
date=first_movie.find("span", class_="date")
date.text

'25 décembre 2020'

In [99]:
age=first_movie.find("div",class_="label kids-label aged-default")
age.text

'à partir de 6 ans'

In [149]:
spectateur=first_movie.find("span",class_="stareval-note")
spectateur.text

'4,2'

In [113]:
titre=films[10].find("a",class_="meta-title-link")
type(titre)

bs4.element.Tag

In [112]:
age=films[10].find("div",class_="label kids-label aged-default")
type(age)

NoneType

## scraper les films Desney+ pour une page
Pour la première page on 12 films, nous allons recuperer les films conditionner par une l'age autorisé pour regarder ce film 

In [164]:
import pandas as pd

In [163]:
Noms=[]
Dates=[]
Age=[]
Spectateur=[]
for film in films:
    condition=film.find("div",class_="label kids-label aged-default")
    if condition is not None:
        
        nom=film.a.text
        Noms.append(nom)
        date=film.find("span", class_="date").text
        Dates.append(date)
        spect=film.find("span",class_="stareval-note").text
        Spectateur.append(spect)
        age=film.find("div",class_="label kids-label aged-default").text
        Age.append(age)



In [165]:
tableau=pd.DataFrame({"Names":Noms,
                     "Dates":Dates,
                     "Age":Age,
                     "Spectateur":Spectateur})
tableau

,Names,Dates,Age,Spectateur
0,Soul,25 décembre 2020,à partir de 6 ans,"4,2"
1,Mulan,30 novembre 2020,à partir de 10 ans,"2,6"
2,"Maman, j'ai raté l'avion",19 décembre 1990,à partir de 8 ans,"3,5"
3,Zootopie,17 février 2016,à partir de 6 ans,"4,0"
4,Le Retour de Mary Poppins,19 décembre 2018,à partir de 8 ans,"3,3"
5,Peter et Elliott le dragon,17 août 2016,à partir de 8 ans,"3,3"
6,Coco,15 novembre 2017,à partir de 8 ans,"4,1"
7,Vice Versa,17 juin 2015,à partir de 6 ans,"4,7"
8,Star Wars : Episode I - La Menace fantôme,13 octobre 1999,à partir de 8 ans,"3,1"
9,La Belle et la Bête,22 mars 2017,à partir de 8 ans,"3,1"


## Controler le taux de requetes envoyées

pourquoi besoin de controler le taux de requetes envoyées?
- en web sraping, le risque c'est d'envoyer un trop grand nombres par seconde et que notre IP soit banni
- l'autre risque est de perturber l'utilité du serveur et donc du site web

Donc pour controler ce taux de requetes on utilise la fonction sleep du module time qui va mettre en pause l'execution de notre boucle pour un certain temps

In [185]:
from time import sleep
from random import randint
from time import time
from IPython.display import clear_output# pour afficher la requete en cours
start=time()# temps du début
requests=0 # la variable requests va compter le nbre de requetes
for k in range(10):
    requests+=1 # on increment la variable requests à chaque passage
    sleep(randint(1,3)) # on fait une pause de 1 à 3 secondes
    end=time()-start # temps écoule pour une requete
    print("requests: {}; frequence : {} requests/s".format(requests,requests/end))  
    clear_output(wait=True)

requests: 10; frequence : 0.5242520297576082 requests/s


In [167]:
urlpage='https://www.allocine.fr/disney/film/tous/?page='

In [168]:
# la fonction get_page donne la liste des pages qu'on souhait scraper
def get_page(urlpage,nombre):
    pages=[]
    for k in range(1,nombre+1):
        res=urlpage+str(k)
        pages.append(res)
    return pages
        

In [193]:
# 1)déclaration des listes vides
nombre=30
pages=get_page(urlpage,nombre)
Names=[]
Dates=[]
Scores=[]
Ages=[]
# 2) preparer l'ecran l'affichage des boucle
start=time()
requests=0

# 3) pour chaque page entre 1 et 5
for page in pages:

    # 4) la requete get()
    reponse=get(page)

    # 5) pouser la boucle de 8 à 15 secondes
    sleep(randint(8,12))

    # 7) afficher le temps de requete
    requests+=1 # on increment la variable requests à chaque passage
    end_url=time()-start
    print("requests:{} ; frequence:{} requete/s".format(requests,requests/end_url))
    clear_output(wait=True)

    # 8) ajouter un avertisement si le code status different de 200
    if reponse.status_code!=200:
        warn('request:{}; status code:{}'.format(requests,reponse.status_code))

    # 9) verifier la condition d'arret
    if requests >50:
        warn("Nombres de requetes trop important")
        break
    # 10) Convertir le contenu HTML response en un objet BeautifulSoup
    page_html=BeautifulSoup(reponse.text,'html.parser')

    # 11) selectionner les 50 films de chaque page (containers)
    movies=page_html.find_all('li',class_='mdl')

    # 12) une boucle qui parcourt tout les containers
    for film in movies:

        # verifions si le film a un metascore
        condition=film.find("div",class_="label kids-label aged-default")    
        if condition is not None:
            # scraper le score du film
            score=film.find("span",class_="stareval-note").text
            Scores.append(score)

            # scraper le nom du film
            nom=film.a.text
            Names.append(nom)

            #scraper la date de sortie du film
            date=film.find("span", class_="date").text
            Dates.append(date)

            # scraper l'age autoriser
            age=film.find("div",class_="label kids-label aged-default").text
            Ages.append(age)
           



requests:30 ; frequence:0.09463791810298744 requete/s


In [194]:
movies_all=pd.DataFrame({"Film":Names,
                            "Dates":Dates,
                            "Ages":Ages,
                            "Scores":Scores,
                        })

In [195]:
print(movies_all.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258 entries, 0 to 257
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Film    258 non-null    object
 1   Dates   258 non-null    object
 2   Ages    258 non-null    object
 3   Scores  258 non-null    object
dtypes: object(4)
memory usage: 8.2+ KB
None


In [196]:
movies_all

,Film,Dates,Ages,Scores
0,Soul,25 décembre 2020,à partir de 6 ans,"4,2"
1,Mulan,30 novembre 2020,à partir de 10 ans,"2,6"
2,"Maman, j'ai raté l'avion",19 décembre 1990,à partir de 8 ans,"3,5"
3,Zootopie,17 février 2016,à partir de 6 ans,"4,0"
4,Le Retour de Mary Poppins,19 décembre 2018,à partir de 8 ans,"3,3"
...,...,...,...,...
253,Le Sortilège de Cendrillon (V),26 septembre 2012,à partir de 6 ans,"2,9"
254,Le Chihuahua de Beverly Hills,25 mars 2009,à partir de 8 ans,"1,8"
255,"Les Muppets, le retour",11 avril 2012,à partir de 8 ans,"3,5"
256,Clochette et la pierre de lune,28 octobre 2009,à partir de 6 ans,"3,7"


In [197]:
movies_all.to_csv('Films_Disney+')# creation d'un fichier csv